# PRO3515 - PFSP CDS


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


### Objetivo
        * Minimizar o tempo total (makespan) na programação de máquinas paralelas (PMP - Parallel Machine Problem),
        utilizando a regra do maior tempo de processamento (LPT - longest processing time)

Importando a biblioteca sugerida

In [20]:
import xlwings as xw

Definindo a classe Job que usaremos no algoritmo

In [21]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness

### Reutilizamos a função disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas

In [22]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    print('n =',n)
    print('m =',m)
    print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i]
        jobs.append(Job(i,pi,0))
    
    return m,n,jobs

Também reutilizamos a de gravar, novamente, com alguns ajustes

In [23]:
def gravaSched(i,jobs,Cmax):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',5]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    print('#','p','C')
    for job in jobs:
        print(job.i, job.p, job.C)
    print('Cmax:',Cmax)

Função que permite calcular o tempo que os jobs se completam

In [24]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

### Procedimento
    1. O método consiste basicamente em resolver “m-1” subproblemas com n jobs e duas máquinas, utilizando o algoritmo de Johnson e dados derivados problema original

    2. O primeiro subproblema consiste em considerar os tempos na primeira máquina e na última máquina

    3. O segundo subproblema consiste em considerar a soma dos tempos nas duas primeiras máquinas e a soma dos tempos nas duas últimas

    4. Genericamente, o subproblema k, consiste em considerar a soma dos tempos nas k primeiras máquinas e a soma dos tempos nas k últimas máquinas (k=1,2,...,m-1)

    5. A melhor sequência dentre as “m-1” geradas é tomada como solução do problema original

In [25]:
def schedJohnson(jobs):
    jobs1=[]
    jobs2=[]
    for job in jobs:
        if job.p[0]<job.p[1]:
            jobs1.append(job)
        else:
            jobs2.append(job)
    
    jobs1.sort(key = lambda job: job.p[0])
    jobs2.sort(key = lambda job: job.p[1], reverse=True)
    
    jobs=jobs1+jobs2
    
    Cmax=calcCmax(2,jobs)     # 2: duas máquinas
    
    return jobs,Cmax

In [41]:
def job_chooser(m,jobs):

    n = len(jobs)
    bestSeq=[]

    bestCmax=9999999999

    for k in range(m-1):

        a=[Job(job.i, [sum(job.p[:k+1]), sum(job.p[m-k-1:])], 0) for job in jobs]     # para cada job, somar os tempos nas k primeiras máquinas

        # b=[Job(job.i, sum(job.p[m-k-1:]), 0) for job in jobs]    # para cada job, somar os tempos nas k últimas máquinas

        seq,cmax = schedJohnson(a)     # aplicar o algoritmo de Johnson com tempos ‘a’ para máquina 1 e ‘b’ para máquina 2

        if cmax<bestCmax:
            bestSeq=seq
            bestCmax=cmax
    
    Final = []
    for item in bestSeq:
        Final.append(jobs[item.i])

    bestCmax = calcCmax(m,Final)

    return Final, bestCmax

## Principal

Abre os dois arquivos que vão ser usados

In [27]:
wb1 = xw.Book("xl05 2 A PFSP Cmax CDS.xlsx")
wb2 = xw.Book()

Caso base pra comprovar o funcionamento do algoritmo implementado

In [45]:
tab1 = [[16, 18, 12],[14,10,11],[13,20,15],[19,15,19],[15,16,16]]
jobs=[]
for i in range(len(tab1)):
    pi=tab1[i]
    jobs.append(Job(i,pi,0))

jobs, Cmax = job_chooser(3,jobs)

print([job.i+1 for job in jobs])
print([job.p for job in jobs])
print(Cmax)

[3, 5, 4, 1, 2]
[[13, 20, 15], [15, 16, 16], [19, 15, 19], [16, 18, 12], [14, 10, 11]]
107


Teste 1:

In [28]:
m,n,jobs=leInst(0)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(0, jobs, Cmax)


n = 20
m = 5
dados = [[58, 60, 88, 14, 46], [34, 33, 17, 67, 15], [54, 25, 93, 60, 2], [19, 84, 25, 47, 8], [24, 45, 25, 16, 11], [21, 69, 93, 22, 71], [64, 43, 48, 83, 39], [37, 43, 18, 46, 13], [26, 57, 34, 5, 18], [82, 14, 98, 27, 14], [84, 43, 96, 67, 81], [75, 87, 45, 11, 44], [73, 27, 32, 9, 79], [24, 94, 39, 23, 95], [49, 54, 8, 70, 97], [27, 61, 55, 87, 35], [98, 8, 90, 15, 55], [51, 93, 80, 86, 35], [83, 87, 25, 94, 80], [12, 80, 12, 49, 38]]

# p C
19 [12, 80, 12, 49, 38] [12, 92, 104, 153, 191]
5 [21, 69, 93, 22, 71] [33, 161, 254, 276, 347]
13 [24, 94, 39, 23, 95] [57, 255, 294, 317, 442]
15 [27, 61, 55, 87, 35] [84, 316, 371, 458, 493]
14 [49, 54, 8, 70, 97] [133, 370, 379, 528, 625]
12 [73, 27, 32, 9, 79] [206, 397, 429, 537, 704]
10 [84, 43, 96, 67, 81] [290, 440, 536, 604, 785]
18 [83, 87, 25, 94, 80] [373, 527, 561, 698, 865]
16 [98, 8, 90, 15, 55] [471, 535, 651, 713, 920]
0 [58, 60, 88, 14, 46] [529, 595, 739, 753, 966]
11 [75, 87, 45, 11, 44] [604, 691, 784, 795, 1

Teste 2:

In [29]:
m,n,jobs=leInst(1)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(1, jobs, Cmax)


n = 20
m = 5
dados = [[96, 12, 54, 58, 95], [68, 46, 39, 53, 90], [26, 87, 24, 1, 13], [23, 50, 2, 24, 34], [66, 63, 16, 90, 9], [83, 38, 41, 21, 34], [31, 13, 17, 55, 71], [63, 76, 77, 70, 95], [19, 80, 86, 46, 54], [44, 78, 3, 94, 66], [89, 58, 17, 27, 28], [49, 78, 63, 52, 28], [32, 41, 76, 86, 86], [77, 34, 5, 73, 91], [33, 29, 8, 67, 36], [24, 17, 37, 79, 44], [85, 92, 21, 54, 7], [49, 42, 13, 51, 67], [84, 90, 27, 85, 86], [98, 67, 13, 55, 57]]

# p C
8 [19, 80, 86, 46, 54] [19, 99, 185, 231, 285]
3 [23, 50, 2, 24, 34] [42, 149, 187, 255, 319]
15 [24, 17, 37, 79, 44] [66, 166, 224, 334, 378]
6 [31, 13, 17, 55, 71] [97, 179, 241, 389, 460]
12 [32, 41, 76, 86, 86] [129, 220, 317, 475, 561]
14 [33, 29, 8, 67, 36] [162, 249, 325, 542, 597]
9 [44, 78, 3, 94, 66] [206, 327, 330, 636, 702]
17 [49, 42, 13, 51, 67] [255, 369, 382, 687, 769]
7 [63, 76, 77, 70, 95] [318, 445, 522, 757, 864]
1 [68, 46, 39, 53, 90] [386, 491, 561, 810, 954]
13 [77, 34, 5, 73, 91] [463, 525, 566, 883, 1045]
1

Teste 3:

In [30]:
m,n,jobs=leInst(2)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(2, jobs, Cmax)


n = 20
m = 5
dados = [[40, 11, 79, 15, 64], [99, 23, 86, 56, 47], [33, 25, 68, 7, 17], [50, 80, 44, 86, 63], [44, 70, 75, 59, 38], [92, 20, 40, 24, 80], [7, 12, 58, 89, 86], [17, 22, 35, 24, 8], [93, 28, 34, 9, 95], [30, 24, 7, 70, 18], [37, 46, 4, 89, 68], [28, 42, 80, 74, 8], [32, 10, 99, 78, 85], [12, 15, 81, 3, 3], [48, 49, 53, 35, 23], [83, 51, 11, 26, 95], [91, 79, 25, 47, 17], [87, 41, 64, 10, 91], [10, 8, 48, 61, 13], [15, 15, 24, 85, 83]]

# p C
6 [7, 12, 58, 89, 86] [7, 19, 77, 166, 252]
18 [10, 8, 48, 61, 13] [17, 27, 125, 227, 265]
19 [15, 15, 24, 85, 83] [32, 47, 149, 312, 395]
12 [32, 10, 99, 78, 85] [64, 74, 248, 390, 480]
10 [37, 46, 4, 89, 68] [101, 147, 252, 479, 548]
0 [40, 11, 79, 15, 64] [141, 158, 331, 494, 612]
3 [50, 80, 44, 86, 63] [191, 271, 375, 580, 675]
15 [83, 51, 11, 26, 95] [274, 325, 386, 606, 770]
17 [87, 41, 64, 10, 91] [361, 402, 466, 616, 861]
8 [93, 28, 34, 9, 95] [454, 482, 516, 625, 956]
5 [92, 20, 40, 24, 80] [546, 566, 606, 649, 1036]
1 [99, 2

Teste 4:

In [31]:
m,n,jobs=leInst(3)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(3, jobs, Cmax)


n = 20
m = 5
dados = [[46, 64, 7, 43, 98], [55, 36, 70, 49, 97], [7, 13, 88, 6, 48], [12, 27, 59, 6, 93], [63, 4, 42, 57, 24], [61, 27, 18, 12, 56], [80, 85, 10, 8, 2], [6, 2, 20, 91, 18], [9, 19, 43, 77, 8], [73, 54, 33, 92, 67], [98, 90, 74, 24, 94], [21, 56, 23, 48, 25], [7, 58, 28, 92, 93], [64, 90, 9, 74, 20], [17, 34, 5, 37, 12], [37, 1, 50, 87, 83], [50, 6, 95, 62, 20], [31, 40, 53, 7, 58], [53, 76, 21, 5, 81], [94, 5, 28, 39, 46]]

# p C
7 [6, 2, 20, 91, 18] [6, 8, 28, 119, 137]
2 [7, 13, 88, 6, 48] [13, 26, 116, 125, 185]
12 [7, 58, 28, 92, 93] [20, 84, 144, 236, 329]
3 [12, 27, 59, 6, 93] [32, 111, 203, 242, 422]
11 [21, 56, 23, 48, 25] [53, 167, 226, 290, 447]
17 [31, 40, 53, 7, 58] [84, 207, 279, 297, 505]
15 [37, 1, 50, 87, 83] [121, 208, 329, 416, 588]
0 [46, 64, 7, 43, 98] [167, 272, 336, 459, 686]
18 [53, 76, 21, 5, 81] [220, 348, 369, 464, 767]
1 [55, 36, 70, 49, 97] [275, 384, 454, 513, 864]
10 [98, 90, 74, 24, 94] [373, 474, 548, 572, 958]
9 [73, 54, 33, 92, 67] [44

Teste 5:

In [32]:
m,n,jobs=leInst(4)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(4, jobs, Cmax)


n = 20
m = 5
dados = [[46, 28, 44, 49, 38], [94, 15, 6, 95, 79], [98, 93, 55, 47, 55], [13, 15, 46, 59, 52], [50, 48, 7, 16, 47], [82, 7, 45, 53, 74], [27, 91, 10, 95, 11], [54, 21, 60, 95, 59], [86, 67, 62, 85, 20], [6, 24, 25, 76, 65], [84, 75, 82, 84, 39], [66, 79, 16, 80, 91], [75, 64, 46, 23, 19], [32, 52, 67, 71, 29], [30, 43, 1, 5, 8], [8, 21, 4, 38, 35], [21, 57, 80, 8, 2], [54, 75, 74, 61, 78], [92, 83, 82, 59, 83], [44, 30, 61, 99, 17]]

# p C
9 [6, 24, 25, 76, 65] [6, 30, 55, 131, 196]
15 [8, 21, 4, 38, 35] [14, 51, 59, 169, 231]
3 [13, 15, 46, 59, 52] [27, 66, 112, 228, 283]
7 [54, 21, 60, 95, 59] [81, 102, 172, 323, 382]
17 [54, 75, 74, 61, 78] [135, 210, 284, 384, 462]
11 [66, 79, 16, 80, 91] [201, 289, 305, 464, 555]
18 [92, 83, 82, 59, 83] [293, 376, 458, 523, 638]
1 [94, 15, 6, 95, 79] [387, 402, 464, 618, 717]
5 [82, 7, 45, 53, 74] [469, 476, 521, 671, 791]
2 [98, 93, 55, 47, 55] [567, 660, 715, 762, 846]
4 [50, 48, 7, 16, 47] [617, 708, 722, 778, 893]
10 [84, 75, 82

Teste 6:

In [33]:
m,n,jobs=leInst(5)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(5, jobs, Cmax)


n = 20
m = 5
dados = [[77, 6, 35, 78, 68], [47, 91, 70, 45, 23], [64, 2, 2, 63, 51], [9, 38, 3, 25, 33], [30, 3, 95, 9, 53], [85, 22, 83, 1, 10], [83, 25, 67, 70, 13], [33, 71, 30, 23, 17], [63, 94, 70, 47, 93], [74, 22, 52, 97, 41], [69, 10, 60, 42, 59], [63, 32, 11, 20, 4], [3, 26, 73, 45, 58], [40, 27, 13, 17, 71], [72, 79, 36, 99, 33], [36, 65, 95, 12, 30], [30, 20, 88, 34, 26], [85, 97, 34, 18, 24], [98, 39, 39, 49, 77], [33, 81, 69, 7, 46]]

# p C
12 [3, 26, 73, 45, 58] [3, 29, 102, 147, 205]
3 [9, 38, 3, 25, 33] [12, 67, 105, 172, 238]
4 [30, 3, 95, 9, 53] [42, 70, 200, 209, 291]
19 [33, 81, 69, 7, 46] [75, 156, 269, 276, 337]
13 [40, 27, 13, 17, 71] [115, 183, 282, 299, 408]
8 [63, 94, 70, 47, 93] [178, 277, 352, 399, 501]
18 [98, 39, 39, 49, 77] [276, 316, 391, 448, 578]
0 [77, 6, 35, 78, 68] [353, 359, 426, 526, 646]
10 [69, 10, 60, 42, 59] [422, 432, 492, 568, 705]
2 [64, 2, 2, 63, 51] [486, 488, 494, 631, 756]
9 [74, 22, 52, 97, 41] [560, 582, 634, 731, 797]
14 [72, 79, 36

Teste 7:

In [34]:
m,n,jobs=leInst(6)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(6, jobs, Cmax)


n = 20
m = 5
dados = [[93, 82, 81, 41, 81], [42, 5, 64, 27, 26], [77, 16, 47, 11, 16], [74, 42, 4, 7, 47], [77, 47, 91, 96, 90], [4, 22, 77, 5, 12], [81, 61, 46, 19, 33], [71, 83, 69, 51, 84], [96, 64, 54, 30, 98], [45, 4, 95, 31, 35], [34, 45, 18, 68, 59], [21, 65, 39, 61, 57], [17, 25, 32, 86, 76], [58, 96, 15, 60, 36], [46, 86, 1, 1, 44], [71, 83, 84, 63, 59], [66, 29, 91, 46, 93], [95, 73, 64, 50, 14], [68, 30, 22, 11, 7], [84, 63, 71, 25, 57]]

# p C
5 [4, 22, 77, 5, 12] [4, 26, 103, 108, 120]
12 [17, 25, 32, 86, 76] [21, 51, 135, 221, 297]
11 [21, 65, 39, 61, 57] [42, 116, 174, 282, 354]
10 [34, 45, 18, 68, 59] [76, 161, 192, 350, 413]
16 [66, 29, 91, 46, 93] [142, 190, 283, 396, 506]
7 [71, 83, 69, 51, 84] [213, 296, 365, 447, 590]
4 [77, 47, 91, 96, 90] [290, 343, 456, 552, 680]
8 [96, 64, 54, 30, 98] [386, 450, 510, 582, 778]
0 [93, 82, 81, 41, 81] [479, 561, 642, 683, 859]
15 [71, 83, 84, 63, 59] [550, 644, 728, 791, 918]
19 [84, 63, 71, 25, 57] [634, 707, 799, 824, 975]
3 [

Teste 8:

In [35]:
m,n,jobs=leInst(7)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(7, jobs, Cmax)


n = 20
m = 5
dados = [[21, 15, 23, 42, 83], [35, 22, 14, 89, 15], [51, 79, 7, 8, 99], [20, 99, 30, 60, 54], [75, 50, 38, 75, 73], [40, 50, 93, 98, 67], [14, 1, 53, 27, 9], [66, 8, 3, 49, 31], [12, 71, 22, 78, 31], [23, 12, 51, 96, 98], [8, 2, 4, 10, 4], [70, 39, 67, 18, 23], [64, 92, 74, 61, 76], [56, 10, 25, 28, 96], [79, 92, 89, 98, 47], [32, 1, 94, 43, 17], [12, 46, 86, 14, 31], [51, 22, 36, 9, 37], [51, 51, 9, 19, 98], [51, 34, 2, 75, 99]]

# p C
8 [12, 71, 22, 78, 31] [12, 83, 105, 183, 214]
16 [12, 46, 86, 14, 31] [24, 129, 215, 229, 260]
3 [20, 99, 30, 60, 54] [44, 228, 258, 318, 372]
0 [21, 15, 23, 42, 83] [65, 243, 281, 360, 455]
9 [23, 12, 51, 96, 98] [88, 255, 332, 456, 554]
5 [40, 50, 93, 98, 67] [128, 305, 425, 554, 621]
2 [51, 79, 7, 8, 99] [179, 384, 432, 562, 720]
18 [51, 51, 9, 19, 98] [230, 435, 444, 581, 818]
19 [51, 34, 2, 75, 99] [281, 469, 471, 656, 917]
13 [56, 10, 25, 28, 96] [337, 479, 504, 684, 1013]
12 [64, 92, 74, 61, 76] [401, 571, 645, 745, 1089]
4 [75, 5

Teste 9:

In [36]:
m,n,jobs=leInst(8)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(8, jobs, Cmax)


n = 20
m = 5
dados = [[66, 73, 83, 96, 24], [69, 99, 84, 96, 95], [26, 4, 65, 62, 59], [92, 14, 5, 40, 5], [55, 20, 45, 74, 31], [4, 24, 7, 79, 41], [57, 71, 59, 4, 46], [52, 71, 71, 88, 87], [94, 55, 18, 70, 25], [43, 17, 12, 77, 16], [44, 62, 63, 80, 64], [96, 49, 77, 71, 89], [18, 9, 17, 41, 38], [91, 92, 13, 74, 80], [16, 84, 21, 53, 88], [32, 93, 65, 30, 25], [45, 22, 14, 94, 16], [80, 5, 89, 28, 69], [30, 26, 73, 45, 1], [71, 71, 57, 35, 44]]

# p C
5 [4, 24, 7, 79, 41] [4, 28, 35, 114, 155]
14 [16, 84, 21, 53, 88] [20, 112, 133, 186, 274]
12 [18, 9, 17, 41, 38] [38, 121, 150, 227, 312]
2 [26, 4, 65, 62, 59] [64, 125, 215, 289, 371]
10 [44, 62, 63, 80, 64] [108, 187, 278, 369, 435]
7 [52, 71, 71, 88, 87] [160, 258, 349, 457, 544]
1 [69, 99, 84, 96, 95] [229, 357, 441, 553, 648]
11 [96, 49, 77, 71, 89] [325, 406, 518, 624, 737]
13 [91, 92, 13, 74, 80] [416, 508, 531, 698, 817]
17 [80, 5, 89, 28, 69] [496, 513, 620, 726, 886]
6 [57, 71, 59, 4, 46] [553, 624, 683, 730, 932]
19 [71,

Teste 10:

In [37]:
m,n,jobs=leInst(9)

jobs, Cmax = job_chooser(m,jobs)

gravaSched(9, jobs, Cmax)


n = 20
m = 5
dados = [[69, 81, 12, 13, 3], [15, 26, 3, 1, 56], [88, 54, 36, 41, 17], [53, 72, 53, 39, 44], [57, 57, 91, 80, 13], [45, 71, 86, 89, 37], [45, 95, 47, 94, 5], [82, 31, 96, 47, 73], [21, 44, 61, 82, 38], [57, 31, 51, 24, 1], [49, 22, 75, 39, 36], [73, 81, 44, 14, 33], [40, 37, 41, 60, 72], [19, 72, 16, 40, 72], [46, 44, 58, 21, 26], [7, 39, 24, 54, 14], [41, 17, 2, 46, 5], [11, 92, 34, 3, 10], [65, 99, 56, 10, 87], [46, 75, 83, 3, 96]]

# p C
15 [7, 39, 24, 54, 14] [7, 46, 70, 124, 138]
1 [15, 26, 3, 1, 56] [22, 72, 75, 125, 194]
13 [19, 72, 16, 40, 72] [41, 144, 160, 200, 272]
8 [21, 44, 61, 82, 38] [62, 188, 249, 331, 369]
12 [40, 37, 41, 60, 72] [102, 225, 290, 391, 463]
19 [46, 75, 83, 3, 96] [148, 300, 383, 394, 559]
18 [65, 99, 56, 10, 87] [213, 399, 455, 465, 646]
7 [82, 31, 96, 47, 73] [295, 430, 551, 598, 719]
3 [53, 72, 53, 39, 44] [348, 502, 604, 643, 763]
5 [45, 71, 86, 89, 37] [393, 573, 690, 779, 816]
10 [49, 22, 75, 39, 36] [442, 595, 765, 818, 854]
11 [73, 

Fecha o workbook de Dados

In [38]:
wb1.close()